# Example attacks against Large Language Models with TextAttack

In this notebook, we will show you how to use Textattack, a library to test the robustness of LLMs.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zangobot/teaching_material/blob/HEAD/Extra-LLMSecurity.ipynb)

In [1]:
!pip install textattack tensorflow tensorflow_hub

  Using cached textattack-0.3.10-py3-none-any.whl.metadata (38 kB)
  Using cached tensorflow-2.18.0-cp310-cp310-macosx_12_0_arm64.whl.metadata (4.0 kB)
  Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
  Using cached editdistance-0.8.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (3.9 kB)
  Using cached flair-0.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached language_tool_python-2.8.1-py3-none-any.whl.metadata (12 kB)
  Using cached lemminflect-0.2.3-py3-none-any.whl.metadata (7.0 kB)
  Using cached lru_dict-1.3.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (4.5 kB)
  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached terminaltables-3.1.10-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached word2number-1.1.zip (9.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached num2words-0.5.13-py3-none-any.whl.metadata (12 kB)
  Using cached more_itertools-10.5.0-py3

In [2]:
import transformers

/Users/zangobot/miniconda3/envs/tai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


An attack in TextAttack consists of four parts:

- **Goal function**: determines if the attack is successful or not: evaluates how well a perturbed attacked_text object is achieving a specified goal. A common goal function is **targeted**/**untargeted classification**, where the attack tries to perturb an input to change its classification.

- **Transformation**: takes a text input and transforms it, for example by replacing words or phrases with similar ones, while trying not to change the meaning. Paraphrase and synonym substitution are two broad classes of transformations.

- **Constraints**: determine whether or not a given transformation is valid. Transformations don't perfectly preserve syntax or semantics, so additional constraints can increase the probability that these qualities are preserved from the source to adversarial example. There are many types of constraints: overlap constraints that measure edit distance, syntactical  constraints check part-of-speech and grammar errors, and semantic constraints like language models and sentence encoders.

- **Search method**: explores the space of possible transformations within the defined constraints and attempt to find a successful perturbation which satisfies the goal function. Some examples are greedy search with word importance ranking, beam search, brute force, and genetic algorithm.


In this way, attacking an NLP model can be framed as a combinatorial search problem. The attacker must search within all potential transformations to find a sequence of transformations that generate a successful adversarial example.

TextAttack provides a set of **Attack Recipes** that assemble attacks from the literature from these four components. We can exploit an attack recipe or build an attack from scratch.


# Example 1: Attack Recipe

In [3]:
from textattack.datasets import HuggingFaceDataset

# load the dataset from HuggingFace
dataset = HuggingFaceDataset("rotten_tomatoes", None, "train")

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/zangobot/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zangobot/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /Users/zangobot/nltk_data...
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/zangobot/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zangobot/nltk_data...
[nltk_data] Downloading package punkt to /Users/zangobot/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Generating test split: 100%|██████████| 1066/1066 [00:00<00:00, 490793.42 examples/s]
textattack: Loading datasets dataset rotten_tomatoes, split train.


In [4]:
# print some examples
print(dataset[0])
print(dataset[1])
print(dataset[2])
print(dataset[3])
print(dataset[4])

(OrderedDict([('text', 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .')]), 1)
(OrderedDict([('text', 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .')]), 1)
(OrderedDict([('text', 'effective but too-tepid biopic')]), 1)
(OrderedDict([('text', 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .')]), 1)
(OrderedDict([('text', "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .")]), 1)


In [5]:
from textattack.models.wrappers import HuggingFaceModelWrapper

# load a pre-trained BERT-based model for sequence classification from the HuggingFace model hub
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "textattack/distilbert-base-uncased-rotten-tomatoes"
)

# load the corresponding tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "textattack/distilbert-base-uncased-rotten-tomatoes"
)

# TextAttack requires both the model and the tokenizer to be wrapped by a ModelWrapper class that implements the forward pass operation given a list of input texts
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

/Users/zangobot/miniconda3/envs/tai/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
print("\nMODEL: ", model)
print("\nTOKENIZER: ", tokenizer)
print("\nWRAPPER: ", model_wrapper)


MODEL:  DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace

Let's use TextFooler, which operates at word-level.

The main idea behind **TextFooler** is to generate adversarial examples by iteratively replacing or modifying words in the input text to maximize the model's prediction error while minimizing the perturbation's perceptibility to human readers. The framework utilizes semantic and syntactic similarity metrics to guide the perturbation process, ensuring that the modified text remains grammatically and semantically coherent.

In [7]:
from textattack.attack_recipes import TextFoolerJin2019

# load the attack
attack = TextFoolerJin2019.build(model_wrapper)

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [12:12<00:00, 657kB/s]  
textattack: Unzipping file /Users/zangobot/.cache/textattack/tmplqssvr3w.zip to /Users/zangobot/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [8]:
print("\nATTACK: ", attack)


ATTACK:  Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis

By printing the attack we can check the definition of the four components: search method, goal function, transformation, constraints.

Let's run the attack.

In [9]:
from textattack import Attacker
from textattack import AttackArgs

attack_args = AttackArgs(num_examples = 10)

attacker = Attacker(attack, dataset, attack_args)

TextAttack attacks iterate through a dataset and, for each correctly predicted sample, search for an adversarial perturbation. If an example is incorrectly predicted to begin with, it is not attacked (Skipped), since the input already fools the model.

In [10]:
attack_results = attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

 10%|█         | 1/10 [00:50<07:32, 50.24s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:50<07:34, 50.52s/it]

[[Positive (86%)]] --> [[Negative (66%)]]

the rock is destined to be the 21st century's new " conan " and that he's going to make a [[splash]] even greater than arnold schwarzenegger , jean-claud van damme or steven segal .

the rock is destined to be the 21st century's new " conan " and that he's going to make a [[splatters]] even greater than arnold schwarzenegger , jean-claud van damme or steven segal .




...

--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (66%)]] --> [[Negative (63%)]]

perhaps no picture ever made has more literally showed that the road to hell is paved with [[good]] intentions .

perhaps no picture ever made has more literally showed that the road to hell is paved with [[decent]] intentions .




[Succeeded / Failed / Skipped / Total] 8 / 0 / 1 / 9:  90%|█████████ | 9/10 [01:36<00:10, 10.72s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (62%)]] --> [[Negative (55%)]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow [[pulls]] it off .

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow [[pulling]] it off .




[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10: 100%|██████████| 10/10 [01:49<00:00, 10.90s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (95%)]] --> [[Negative (55%)]]

take [[care]] of my cat [[offers]] a [[refreshingly]] different slice of [[asian]] [[cinema]] .

take [[zorg]] of my cat [[suggests]] a [[thankfully]] different slice of [[asiatic]] [[flick]] .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 13.09% |
| Average num. words per input: | 19.5   |
| Avg num queries:              | 81.44  |
+-------------------------------+--------+


To visualize the results, we are logging AttackResult objects using a CSVLogger. This logger stores all attack results in a dataframe, which we can easily access and display. Since we set color_method to 'html', the attack results will display their differences, in color, in HTML. Using IPython utilities and pandas

In [11]:
import pandas as pd
from textattack.loggers import CSVLogger
from textattack.attack_results import SuccessfulAttackResult

# pd.options.display.max_colwidth = (
#     480  # increase colum width so we can actually read the examples
# )

logger = CSVLogger(color_method="html")

for result in attack_results:
    if isinstance(result, SuccessfulAttackResult):
        logger.log_attack_result(result)

from IPython.core.display import display, HTML

results = pd.DataFrame.from_records(logger.row_list)
display(HTML(results[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Logging to CSV at path results.csv
/var/folders/gz/vw3qnvfj4pb_6v3wt_l5h5480000gn/T/ipykernel_16528/3313485548.py:15: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,original_text,perturbed_text
0,"the rock is destined to be the 21st century's new "" conan "" and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .","the rock is destined to be the 21st century's new "" conan "" and that he's going to make a splatters even greater than arnold schwarzenegger , jean-claud van damme or steven segal ."
1,"the gorgeously elaborate continuation of "" the lord of the rings "" trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .","the gorgeously elaborate continuation of "" the lord of the rings "" trilogy is too huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth ."
2,"if you sometimes like to go to the movies to have fun , wasabi is a good place to start .","if you sometimes like to go to the movies to have fun , wasabi is a opportune place to start ."
3,"emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .","emerges as something rare , an issue movie that's so franc and earnestly observed that it doesn't meaning enjoy one ."
4,the film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .,the film gives some sizable premonition into the neurotic geist of all comics -- even those who have reached the total top of the plaything .
5,offers that rare combination of entertainment and education .,offers that scant combination of entertainment and education .
6,perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .,perhaps no picture ever made has more literally showed that the road to hell is paved with decent intentions .
7,steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulls it off .,steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulling it off .
8,take care of my cat offers a refreshingly different slice of asian cinema .,take zorg of my cat suggests a thankfully different slice of asiatic flick .


# Example 2: Attack from scratch


In [12]:
# Load the dataset
from textattack.datasets import HuggingFaceDataset

dataset = HuggingFaceDataset("ag_news", None, "test")

Generating test split: 100%|██████████| 7600/7600 [00:00<00:00, 1549443.95 examples/s]
textattack: Loading datasets dataset ag_news, split test.


In [13]:
# Load the model
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained(  # loads a pre-trained BERT-based model for sequence classification from the HuggingFace model hub
    "textattack/bert-base-uncased-ag-news"
)
tokenizer = transformers.AutoTokenizer.from_pretrained( # loads the corresponding tokenizer for the BERT model
    "textattack/bert-base-uncased-ag-news"
)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

/Users/zangobot/miniconda3/envs/tai/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Banana word swap
We're going to try a very simple transformation: one that replaces any given word with the word 'banana'. In TextAttack, there's an abstract WordSwap class that handles the heavy lifting of breaking sentences into words and avoiding replacement of stopwords. We can extend WordSwap and implement a single method, _get_replacement_words, to indicate to replace each word with 'banana'. 🍌

In [14]:
from textattack.transformations import WordSwap

class BananaWordSwap(WordSwap):
    """Transforms an input by replacing any word with 'banana'."""

    def _get_replacement_words(self, word):
        """Returns a list with one item, since `_get_replacement_words` is intended to return a list of candidate replacement words."""
        return ["banana"]

In [15]:
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import (
    RepeatModification,   # constraint disallowing the modification of words which have already been modified
    StopwordModification, # constraint disallowing the modification of stopwords
)
from textattack import Attack

# Create the goal function using the model
goal_function = UntargetedClassification(model_wrapper)

# Use Banana word swap class as the attack transformation
transformation = BananaWordSwap()

# Avoid modification of already modified indices and stopwords
constraints = [RepeatModification(), StopwordModification()]

# Use the Greedy search method
search_method = GreedySearch()

# Create the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [16]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [17]:
from tqdm import tqdm
from textattack import Attacker
from textattack import AttackArgs

attack_args = AttackArgs(num_examples=10)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:15<02:20, 15.57s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Business (100%)]] --> [[World (89%)]]

Fears for T N [[pension]] after [[talks]] [[Unions]] representing [[workers]] at Turner   Newall say they are '[[disappointed']] after talks with stricken parent firm Federal [[Mogul]].

Fears for T N [[banana]] after [[banana]] [[banana]] representing [[banana]] at Turner   Newall say they are '[[banana]] after talks with stricken parent firm Federal [[banana]].




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [02:07<08:30, 63.79s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Sci/tech (100%)]] --> [[World (64%)]]

The Race is On: Second Private [[Team]] Sets Launch [[Date]] for [[Human]] [[Spaceflight]] ([[SPACE]].[[com]]) [[SPACE]].[[com]] - [[TORONTO]], [[Canada]] -- [[A]] [[second]]\[[team]] of rocketeers [[competing]] for the  #36;10 million Ansari X [[Prize]], a [[contest]] for\[[privately]] funded [[suborbital]] [[space]] [[flight]], has officially [[announced]] the first\[[launch]] date for its [[manned]] rocket.

The Race is On: Second Private [[banana]] Sets Launch [[banana]] for [[banana]] [[banana]] ([[banana]].[[banana]]) [[banana]].[[banana]] - [[banana]], [[banana]] -- [[banana]] [[banana]]\[[banana]] of rocketeers [[banana]] for the  #36;10 million Ansari X [[banana]], a [[banana]] for\[[banana]] funded [[banana]] [[banana]] [[banana]], has officially [[banana]] the first\[[banana]] date for its [[banana]] rocket.




...

--------------------------------------------- Result 8 ---------------------------------------------
[[Sci/tech (100%)]] --> [[[FAILED]]]

FOAFKey: FOAF, PGP, Key Distribution, and Bloom Filters \\FOAF/LOAF  and bloom filters have a lot of interesting properties for social\network and whitelist distribution.\\I think we can go one level higher though and include GPG/OpenPGP key\fingerpring distribution in the FOAF file for simple web-of-trust based key\distribution.\\What if we used FOAF and included the PGP key fingerprint(s) for identities?\This could mean a lot.  You include the PGP key fingerprints within the FOAF\file of your direct friends and then include a bloom filter of the PGP key\fingerprints of your entire whitelist (the source FOAF file would of course need\to be encrypted ).\\Your whitelist would be populated from the social network as your client\discovered new identit ...\\




[Succeeded / Failed / Skipped / Total] 7 / 2 / 0 / 9:  90%|█████████ | 9/10 [24:41<02:44, 164.59s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Sci/tech (98%)]] --> [[World (100%)]]

[[E-mail]] scam targets police chief Wiltshire Police warns about "[[phishing]]" after its fraud squad chief was targeted.

[[banana]] scam targets police chief Wiltshire Police warns about "[[banana]]" after its fraud squad chief was targeted.




[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [24:45<00:00, 148.56s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Sci/tech (98%)]] --> [[World (77%)]]

Card fraud unit nets 36,000 cards In its first two years, the UK's dedicated [[card]] fraud unit, has recovered 36,000 stolen cards and 171 arrests - and estimates it saved 65m.

Card fraud unit nets 36,000 cards In its first two years, the UK's dedicated [[banana]] fraud unit, has recovered 36,000 stolen cards and 171 arrests - and estimates it saved 65m.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 18.71% |
| Average num. words per input: | 63.0   |
| Avg num queries:     

In [18]:
import pandas as pd
from textattack.loggers import CSVLogger
from textattack.attack_results import SuccessfulAttackResult

# pd.options.display.max_colwidth = (
#     480  # increase colum width so we can actually read the examples
# )

logger = CSVLogger(color_method="html")

for result in attack_results:
    if isinstance(result, SuccessfulAttackResult):
        logger.log_attack_result(result)

from IPython.core.display import display, HTML

results = pd.DataFrame.from_records(logger.row_list)
display(HTML(results[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Logging to CSV at path results.csv
textattack: CSVLogger exiting without calling flush().
/var/folders/gz/vw3qnvfj4pb_6v3wt_l5h5480000gn/T/ipykernel_16528/3313485548.py:15: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,original_text,perturbed_text
0,Fears for T N pension after talks Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.,Fears for T N banana after banana banana representing banana at Turner Newall say they are 'banana after talks with stricken parent firm Federal banana.
1,"The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.","The Race is On: Second Private banana Sets Launch banana for banana banana (banana.banana) banana.banana - banana, banana -- banana banana\banana of rocketeers banana for the #36;10 million Ansari X banana, a banana for\banana funded banana banana banana, has officially banana the first\banana date for its banana rocket."
2,"Ky. Company Wins Grant to Study Peptides (AP) AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.","Ky. Company Wins Grant to banana banana (banana) banana - banana company banana by a banana banana at the banana of Louisville won a grant to develop a method of producing better banana, which are short chains of banana banana, the building blocks of banana."
3,"Prediction Unit Helps Forecast Wildfires (AP) AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.","banana Unit Helps banana Wildfires (AP) banana - It's barely dawn when Mike Fitzpatrick banana his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar."
4,"Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure.","Calif. Aims to Limit Farm-Related banana (AP) AP - Southern California's banana agency went after banana of the banana variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure."
5,"Open Letter Against British Copyright Indoctrination in Schools The British Department for Education and Skills (DfES) recently launched a ""Music Manifesto"" campaign, with the ostensible intention of educating the next generation of British musicians. Unfortunately, they also teamed up with the music industry (EMI, and various artists) to make this popular. EMI has apparently negotiated their end well, so that children in our schools will now be indoctrinated about the illegality of downloading music.The ignorance and audacity of this got to me a little, so I wrote an open letter to the DfES about it. Unfortunately, it's pedantic, as I suppose you have to be when writing to goverment representatives. But I hope you find it useful, and perhaps feel inspired to do something similar, if or when the same thing has happened in your area.","Open banana Against banana banana Indoctrination in Schools The banana Department for Education and Skills (DfES) banana banana a ""banana banana"" campaign, with the ostensible banana of banana the banana banana of banana banana. banana, they also teamed up with the banana industry (banana, and banana banana) to make this popular. banana has banana banana their end well, so that banana in our schools will now be indoctrinated about the illegality of banana music.The ignorance and audacity of this got to me a little, so I wrote an op

Looks like some examples needed only a couple "banana"s, while others needed up to 17 "banana" substitutions to change the class score.